In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as ply
%matplotlib inline

In [4]:
# read all the words
words= open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [8]:
# build vocabulary of characters and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [10]:
# build the dataset

block_size = 3
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] #crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [12]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [16]:
C = torch.randn((27,2))

In [18]:
C[5]

tensor([-0.2529, -0.8903])

In [23]:
# one hot has to convert to float as it returns Long Int64
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.2529, -0.8903])

In [26]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [29]:
# the inputs to this layer will be 3 * 2, as we have 2d embeddings and 3 of them.
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [31]:
# when -1 is used pytorch will infer what this should be.
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [33]:
h

tensor([[-0.7692, -0.9999,  0.8313,  ..., -0.9651,  0.9497, -0.1309],
        [-0.6724, -0.9982,  0.2540,  ..., -0.9775,  0.9845,  0.3139],
        [-0.9988, -1.0000,  0.9853,  ..., -0.9694,  0.8013,  0.1603],
        ...,
        [-0.9966, -0.9999,  0.9997,  ..., -0.9622,  0.3316, -0.9593],
        [-0.9979, -0.7365,  0.9964,  ..., -0.8413,  0.6245, -0.8815],
        [-0.7123, -0.2815,  0.6165,  ..., -0.9877,  0.9864, -0.9873]])

In [35]:
# explanantion for 'h' process

emb @ W1 + b1 # won't work as emb is [32, 3, 2] and W1 is [6, 100]

# opt 1 : torch.cat()
# won't generalize if we want to change the block size
# creates new tensors and memory, inefficient 
torch.cat([emb[:, 0, :], emb[:, 1,:], emb[:, 2, :]], 1)

# opt 2: torch.cat() with torch.unbind()
torch.cat(torch.unbind(emb, 1), 1)

# opt 3: .view() 'best'


RuntimeError: mat1 and mat2 shapes cannot be multiplied (96x2 and 6x100)

In [37]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [39]:
logits = h @ W2 + b2

In [41]:
logits.shape

torch.Size([32, 27])

In [43]:
counts = logits.exp()

In [46]:
prob = counts / counts.sum(1, keepdims=True)

In [48]:
prob.shape

torch.Size([32, 27])

In [51]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(13.1875)

In [53]:
#  --------- make more respectable ------------ #

In [54]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [58]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [60]:
sum(p.nelement() for p in parameters) # number of total parameters

3481

In [63]:
# forward pass
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.7697)

In [64]:
F.cross_entropy(logits, Y)

tensor(17.7697)